### Environment Setup

In [8]:
!git clone https://github.com/muhammadummerr/Intelligent-Receipt.git
%cd Intelligent-Receipt

Cloning into 'Intelligent-Receipt'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 74 (delta 11), reused 73 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 71.20 KiB | 2.85 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/kaggle/working/Intelligent-Receipt/Intelligent-Receipt


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 79.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127

In [10]:
!pip uninstall -y scikit-learn


Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2


### Train LayoutLMv3 — Receipt Field Extraction

In [11]:
%env PYTHONPATH=./src

!python -u -m receipt_ie.models.training.train_layoutlmv3

env: PYTHONPATH=./src
2025-10-19 18:18:47.804569: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760897927.827791     214 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760897927.835128     214 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
preprocessor_config.json: 100%|████████████████| 275/275 [00:00<00:00, 2.14MB/s]
tokenizer_config.json: 1.14kB [00:00, 6.23MB/s]
vocab.json: 899kB [00:00, 22.3MB/s]
merges.txt: 456kB [00:00, 87.8MB/s]
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted 

### Upload Trained Model to Hugging Face

In [8]:
from huggingface_hub import HfApi, upload_folder, create_repo
from kaggle_secrets import UserSecretsClient

# --- Loading Hugging Face token ---
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# --- Repo details ---
repo_id = "muhammadummerrr/layoutlmv3-receipt-epochs-20"

# --- Initialize API and create repo ---
api = HfApi()
api.create_repo(repo_id=repo_id, private=False, token=hf_token, exist_ok=True)

# --- Upload fine-tuned model folder ---
upload_folder(
    folder_path="/kaggle/temp/outputs_layoutlmv3/final_model",
    repo_id=repo_id,
    token=hf_token,
)

print(f"Uploaded fine-tuned LayoutLMv3 model to https://huggingface.co/{repo_id}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Augment Watermark dataset

In [3]:
!python -m src.receipt_ie.models.watermark.augment_watermark_dataset \
  --input_dir /kaggle/input/wm-dataset/watermark-dataset \
  --output_dir /kaggle/working/receipt-watermark-augmented \
  --split_ratio 0.8 \
  --aug_per_image 3


/kaggle/working/Intelligent-Receipt/src/receipt_ie/models/watermark/augment_watermark_dataset.py:40: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=85, quality_upper=100, p=0.4),
/kaggle/working/Intelligent-Receipt/src/receipt_ie/models/watermark/augment_watermark_dataset.py:42: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 20), p=0.3),
📸 Processing clean -> train (68 images)...
100%|███████████████████████████████████████████| 68/68 [00:32<00:00,  2.08it/s]
📸 Processing clean -> test (17 images)...
100%|███████████████████████████████████████████| 17/17 [00:05<00:00,  2.97it/s]
📸 Processing watermark -> train (68 images)...
100%|███████████████████████████████████████████| 68/68 [01:04<00:00,  1.05it/s]
📸 Processing watermark -> test (17 images)...
100%|███████████████████████████████████████████| 17/17 [00:26<00:00,  1.54s/it]
 Augmented 

### Train ViT Watermark Detector

In [6]:
!python -m src.receipt_ie.models.watermark.train_vit_watermark_classifier \
  --data_root /kaggle/working/receipt-watermark-augmented \
  --out_dir /kaggle/working/wm_vit_output \
  --epochs 10 \
  --batch 32 \
  --lr 5e-5


2025-10-19 22:27:33.404425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760912853.427243     153 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760912853.434097     153 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Found: train=544 val=136
config.json: 100%|█████████████████████████████| 502/502 [00:00<00:00, 3.74MB/s]
model.safetensors: 100%|██████████████████████| 346M/346M [00:01<00:00, 205MB/s]
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream ta

### Upload ViT Model to Hugging Face

In [ ]:
# --- Repo details for the ViT watermark model ---
api = HfApi()
repo_id = "muhammadummerrr/vit-water-mark-detector-v1-extended"
api.create_repo(repo_id=repo_id, private=False, token=hf_token, exist_ok=True)
upload_folder(
    folder_path="/kaggle/working/wm_vit_advanced_out",  # your model output folder
    repo_id=repo_id,
    token=hf_token,
)

print(f"Uploaded watermark classifier to https://huggingface.co/{repo_id}")


In [15]:
!git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 1.91 KiB | 980.00 KiB/s, done.
From https://github.com/muhammadummerr/Intelligent-Receipt
   f6d2269..93f5272  main       -> origin/main
Updating f6d2269..93f5272
Fast-forward
 src/receipt_ie/pipelines/run_pipeline.py | 38 +++++++++++++++++++-------------
 1 file changed, 23 insertions(+), 15 deletions(-)


### End-to-End Inference Pipeline

In [22]:
!python -m src.receipt_ie.pipelines.run_pipeline \
  --image_path /kaggle/input/receipt-dataset/test/img/X51005587261.jpg \
  --box_dir /kaggle/input/receipt-dataset/test/box \
  --out_path /kaggle/working/X51005230605_result.json \
  --provider groq \
  --model llama-3.3-70b-versatile


2025-10-20 13:13:45.965366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760966025.989811     567 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760966025.997502     567 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🔍 Processing: X51005587261.jpg
🛡️ Checking for watermarks or tampering...
📄 Found 1 valid receipts for inference.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
🧾 Extracted: {'company': 'CHOP', 'date': '23/03/17', 'address': 'COMPANY', 'total': '30.00'}
[INFO] LLM initialized — provider=groq, mo

In [ ]:
/kaggle/input/receipt-dataset/test/img/X51005447844.jpg